# Debug

Debug the problems

## Preamble

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import bilby

bilby_keys = ["chirp_mass",
              "mass_ratio",
              "chi_1",
              "chi_2",
              "luminosity_distance",
              "geocent_time",
              "phase",
              "theta_jn",
              "psi",
              "ra",
              "dec"
]

# Replicate the steps of parallel bilby

### Load the datadump

In [2]:
def extract_injection_from_datadump(datadump_file: str,
                                    verbose: bool=False):
    
    print(f"Extracting the injection data from data file: {datadump_file}")
    
    # Open and load
    with open(datadump_file, "rb") as f:
        data = pickle.load(f)
        
        print(data.keys())
        ifo_list = data["ifo_list"]
        injection_parameters = data["injection_parameters"]
    
    # Save info for each ifo
    for ifo in ifo_list:
        # Get the strain
        strain_data = ifo.strain_data # InterferometerStrainData
        _frequency_domain_strain = strain_data._frequency_domain_strain
        _times_and_frequencies = strain_data._times_and_frequencies # CoupledTimeAndFrequencySeries object
        
        freqs = _times_and_frequencies.frequency_array
        real_strain = _frequency_domain_strain.real
        imag_strain = _frequency_domain_strain.imag
        
        # Get the psd values
        psd_values = ifo.power_spectral_density._PowerSpectralDensity__power_spectral_density_interpolated(freqs)
        
        # Assert all have same length
        assert len(freqs) == len(real_strain) == len(imag_strain) == len(psd_values), "Some shape mismatch"
            
        print(f"Saving {ifo} data to file")
        np.savez(f"{ifo.name}_data.npz", freqs=freqs, real_strain=real_strain, imag_strain=imag_strain, psd_values=psd_values)
        
    # Save the injection parameters
    with open(f"injection_parameters.pkl", "wb") as f:
        pickle.dump(injection_parameters, f)
        
    return 

In [3]:
datadump_file = "../11d_runs/outdir/out_bbh_0/data/bbh_0_data_dump.pickle"
extract_injection_from_datadump(datadump_file=datadump_file, verbose=True)

Extracting the injection data from data file: ../11d_runs/outdir/out_bbh_0/data/bbh_0_data_dump.pickle
dict_keys(['waveform_generator', 'ifo_list', 'prior_file', 'args', 'data_dump_file', 'meta_data', 'injection_parameters'])
Saving Interferometer(name='H1', power_spectral_density=PowerSpectralDensity(psd_file='../psds/psd.txt', asd_file='None'), minimum_frequency=20.0, maximum_frequency=2048.0, length=4.0, latitude=46.45514666666667, longitude=-119.4076571388889, elevation=142.554, xarm_azimuth=125.9994, yarm_azimuth=215.9994, xarm_tilt=-0.0006195, yarm_tilt=1.25e-05) data to file
Saving Interferometer(name='L1', power_spectral_density=PowerSpectralDensity(psd_file='../psds/psd.txt', asd_file='None'), minimum_frequency=20.0, maximum_frequency=2048.0, length=4.0, latitude=30.562894333333332, longitude=-90.77424038888887, elevation=-6.574, xarm_azimuth=197.7165, yarm_azimuth=287.7165, xarm_tilt=-0.0003121, yarm_tilt=-0.0006107) data to file
Saving Interferometer(name='V1', power_spectra

### Ifos

### Likelihood


In [4]:
likelihood_kwargs = dict(
    interferometers=ifo_list,
    waveform_generator=waveform_generator,
    priors=priors,
    phase_marginalization=args.phase_marginalization,
    distance_marginalization=args.distance_marginalization,
    distance_marginalization_lookup_table=args.distance_marginalization_lookup_table,
    time_marginalization=args.time_marginalization,
    reference_frame=args.reference_frame,
    time_reference=args.time_reference,
)

NameError: name 'ifo_list' is not defined

In [ ]:
from parallel_bilby.analysis.likelihood import setup_likelihood

/home/urash/twouters/miniconda3/envs/parallel_bilby/lib/python3.10/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(True)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS
15:30 bilby_pipe WARNING : You do not have nds2 (python-nds2-client) installed. You may  experience problems accessing interferometer data.
15:30 bilby_pipe WARNING : You do not have LDAStools.frameCPP (python-ldas-tools-framecpp) installed. You may experience problems accessing interferometer da

In [ ]:
likelihood = setup_likelihood(
            interferometers=ifo_list,
            waveform_generator=waveform_generator,
            priors=priors,
            args=args,
        )

15:30 bilby INFO    : Initialise likelihood <class 'bilby.gw.likelihood.base.GravitationalWaveTransient'> with kwargs: 
{'interferometers': [Interferometer(name='H1', power_spectral_density=PowerSpectralDensity(psd_file='../psds/psd.txt', asd_file='None'), minimum_frequency=20.0, maximum_frequency=2048.0, length=4.0, latitude=46.45514666666667, longitude=-119.4076571388889, elevation=142.554, xarm_azimuth=125.9994, yarm_azimuth=215.9994, xarm_tilt=-0.0006195, yarm_tilt=1.25e-05), Interferometer(name='L1', power_spectral_density=PowerSpectralDensity(psd_file='../psds/psd.txt', asd_file='None'), minimum_frequency=20.0, maximum_frequency=2048.0, length=4.0, latitude=30.562894333333332, longitude=-90.77424038888887, elevation=-6.574, xarm_azimuth=197.7165, yarm_azimuth=287.7165, xarm_tilt=-0.0003121, yarm_tilt=-0.0006107), Interferometer(name='V1', power_spectral_density=PowerSpectralDensity(psd_file='../psds/psd_virgo.txt', asd_file='None'), minimum_frequency=20.0, maximum_frequency=2048.

## TODO: Evaluate the likelihood on the jim samples

In [ ]:
LOG_PRIOR_VALUE = -18.294883803860813

jim_file = "./working_3/jim/results_production.npz"
data = np.load(jim_file)
chains = data["chains"]
chains = np.reshape(chains, (-1, chains.shape[-1]))
log_prob = data["log_prob"]
log_prob = np.reshape(log_prob, (-1, ))

jim_naming = ["M_c", "q", "s1_z", "s2_z", "d_L", "t_c", "phase_c", "iota", "psi", "ra", "dec"]

# translating keys from bilby to jim
translation = {"chirp_mass": "M_c",
               "mass_ratio": "q",
               "chi_1": "s1_z",
                "chi_2": "s2_z",
                "luminosity_distance": "d_L",
                "geocent_time": "t_c",
                "dec": "dec",
                "ra": "ra",
                "theta_jn": "iota",
                "psi": "psi",
                "phase": "phase_c",
                "geocent_time": "t_c"
               }

bilby_keys = translation.keys()
def get_jim_vals(idx):
    """Get the values as dictionary for bilby consumption at specific idx in the jim chains"""
    new_vals = {}
    for name in bilby_keys:
        if name in translation:
            new_vals[name] = chains[idx, jim_naming.index(translation[name])]
        else:
            raise ValueError(f"Key {name} not found in translation")
        
    return new_vals

In [ ]:
idx_list = np.arange(0, 100, 1)

bilby_L = []
jim_L = []

for idx in idx_list:
    new_vals = get_jim_vals(idx)
    likelihood.parameters.update(new_vals)
    bilby_L.append(likelihood.log_likelihood())
    jim_L.append(log_prob[idx])
    
bilby_L = np.array(bilby_L)
jim_L = np.array(jim_L)
diff = bilby_L - jim_L

Thibeau: get params
Thibeau: model_strain is
{'plus': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]), 'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])}
Thibeau: get params
Thibeau: model_strain is
{'plus': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]), 'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])}
Thibeau: get params
Thibeau: model_strain is
{'plus': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]), 'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])}
Thibeau: get params
Thibeau: model_strain is
{'plus': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]), 'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])}
Thibeau: get params
Thibeau: model_strain is
{'plus': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]), 'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])}
Thibeau: get params
Thibeau: model_strain is
{'plus': array([0.+0

In [ ]:
print(np.mean(diff))

-44.456006325160075


# ==== To do ====

## Setup

In [ ]:
# plt.hist(bilby_L, bins = 20)
# plt.hist(jim_L, bins = 20)
# plt.show()

In [ ]:
# plt.hist(diff, bins = 20)
# plt.show()

## Interferometers

In [ ]:
# for ifo in ifos:
#     # print(ifo.meta_data)
#     print(ifo.strain_data.duration)
#     break

In [ ]:
# from bilby.gw.detector import Interferometer
# from bilby.gw.detector.networks import InterferometerList

In [ ]:
# # new_ifo = Interferometer()
# new_ifos = InterferometerList(["H1", "L1", "V1"])
# new_ifos